In [ ]:
# REQ for jupyter file
import os
os.chdir('/home/gutmanb/ftd/fhir/ftd/pipeline-utils')


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from dbt_pipeline_utils.scripts.helpers.common import *
from dbt_pipeline_utils import logger
from scripts.cleaning_helpers import *
from scripts.cleaning_common import *


In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None)  # None means no limit on column width

display(HTML("<style>.output_area pre { white-space: pre-wrap; word-wrap: break-word; }</style>"))

In [ ]:
df_path = "./data/input/cmg-bh/CMG_data_model_16July2020 - Fields and definitions.csv"
column_map = DD_FORMATS['pipeline_format'] 

In [ ]:
# import the data file column names
new_column_names = ['tables',
                    'variable_name',
                    'description',
                    'data_type',
                    'field_priority',
                    'enumerations',
                    'multi_value_delimeter',
                    'examples',
                    '1']

df = pd.read_csv(df_path, names=new_column_names, header=0, index_col=None)

In [ ]:
expand_rows = {}

expand_rows = explode_rows_on_col_value(df,'tables',', ')
print(expand_rows.columns)


In [ ]:
# Clean tables col

# distinct_tables = expand_rows['tables'].unique()

expand_rows['tables']=expand_rows['tables'].str.replace('and Discovery Tables', 'Discovery') \
                                           .str.replace('and Sequencing', 'Sequencing')

distinct_tables = expand_rows['tables'].unique()

print("EXPECTED VALS: 'Subject', 'Sample', 'Discovery', 'Family', 'Sequencing'")
distinct_tables

In [ ]:
# clean enumerations

# view = expand_rows[~expand_rows['enumerations'].isnull()]

expand_rows['enumerations'] = expand_rows['enumerations'].str.replace("\n",';')
# view = expand_rows[~expand_rows['enumerations'].isnull()]

expand_rows['enums_valid'] = expand_rows['enumerations'].str.match(r'^.+(;.+)+$')
expand_rows['enums_valid'] = expand_rows['enums_valid'].fillna(True).astype(bool)


view = expand_rows[~expand_rows['enums_valid']]
view.head(5)


In [ ]:
# Separate dds
print(distinct_tables)
print(DD_FORMATS['pipeline_format'])


cols = ['variable_name','description','data_type','min','max','units','enumerations','comment']

expand_rows['min'] = None
expand_rows['max'] = None
expand_rows['units'] = None
expand_rows['comment'] = None


subject_dd = expand_rows[cols][expand_rows['tables'] == 'Subject']
sample_dd = expand_rows[cols][expand_rows['tables'] == 'Sample']
discovery_dd = expand_rows[cols][expand_rows['tables'] == 'Discovery']
family_dd = expand_rows[cols][expand_rows['tables'] == 'Family']
sequencing_dd = expand_rows[cols][expand_rows['tables'] == 'Sequencing']


In [ ]:

dds= [subject_dd,sample_dd,discovery_dd,family_dd,sequencing_dd]
output_path = 'data/output/cmg/bh'

subject_dd.to_csv(f'{output_path}_subject_dd.csv', index=None)
subject_dd.to_csv(f'{output_path}_sample_dd.csv', index=None)
subject_dd.to_csv(f'{output_path}_discovery_dd.csv', index=None)
subject_dd.to_csv(f'{output_path}_family_dd.csv', index=None)
subject_dd.to_csv(f'{output_path}_sequencing_dd.csv', index=None)

print(f'DDs created in data outputs.')